### Calibration températures de Tourterelle en Infrarouge

In [36]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import sys

sys.path.append('../drone_IR')
from open_IRimage_dji import open_IR_image


In [37]:
path2csv = r"F:\bicwin26\Summary\calibration_IR\calibration_Tourterelle_0209.csv"

In [ ]:
csvdata = pd.read_csv(path2csv, sep=';')

def convert_to_float(value):
    if type(value) == float:
        return value
    else:
        return float(value.replace(',', '.'))

T_IR_celsius = np.zeros(len(csvdata['T_IR_celsius'].values))
T_ref_SBE56_num1_celsius = np.zeros(len(csvdata['T_ref_SBE56_num1_celsius'].values))
T_ref_SBE56_num2_celsius = np.zeros(len(csvdata['T_ref_SBE56_num2_celsius'].values))

for i in range(len(csvdata['T_IR_celsius'].values)):
    
    T_IR_celsius[i] = convert_to_float(csvdata['T_IR_celsius'].values[i])
    T_ref_SBE56_num1_celsius[i] = convert_to_float(csvdata['T_ref_SBE56_num1_celsius'].values[i])
    T_ref_SBE56_num2_celsius[i] = convert_to_float(csvdata['T_ref_SBE56_num2_celsius'].values[i])

surface = csvdata['surface'].values


In [50]:
%matplotlib qt

default_emissivity = 0.95

Tref_glace = []
T_IR_default_glace = []
Tref_neige = []
T_IR_default_neige = []


for i in range(len(surface)):
    if surface[i]=='glace':
        Tref_glace.append(np.nanmean([T_ref_SBE56_num1_celsius[i], T_ref_SBE56_num2_celsius[i]]))
        T_IR_default_glace.append(T_IR_celsius[i])
    elif surface[i]=='neige':
        Tref_neige.append(np.nanmean([T_ref_SBE56_num1_celsius[i], T_ref_SBE56_num2_celsius[i]]))
        T_IR_default_neige.append(T_IR_celsius[i])

Tref_glace = np.array(Tref_glace)
T_IR_default_glace = np.array(T_IR_default_glace)
Tref_neige = np.array(Tref_neige)
T_IR_default_neige = np.array(T_IR_default_neige)
plt.figure(figsize=(8, 6))
plt.scatter(T_IR_default_glace, Tref_glace, label='Données de calibration glace', color='blue')
plt.scatter(T_IR_default_neige, Tref_neige, label='Données de calibration neige', color='red')
plt.xlabel('Température mesurée par IR (emissivité default) (°C)')
plt.ylabel('Température de référence (°C)')
plt.xlim(-20, 0)
plt.ylim(-20, 0)
plt.show()

### Estimation de l'émissivité pour chacune des surfaces observées

On relie La température, l'émissivité, et la puissance infrarouge émise par unité de surface par la relation :

$ M = \epsilon \cdot \sigma \cdot T^4 $

Avec $M$ la puissance infrarouge émise par unité de surface.

Et dans le cas "default", la conversion est (faussement) faite comme ceci :
$ M_{default} =  0.95 \cdot \sigma \cdot T^4 $

